In [49]:
import numpy as np 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import control.matlab as control_
import control
import math 
from sympy import symbols, solve

def pos(lst):
    return [x for x in lst if x > 0] or None

def jitter(inps):
    ICP,PM,FC= inps
    pi=math.pi
    X= symbols('x')
    PM=math.radians(PM)
    eq=((X-(1/X))/2)-(np.tan(PM))
    x=float(pos(solve(eq,X))[0])
    #Assigning some constants 
    Fref=160e6
    Fc=(FC)
    Icp=(ICP)
    R_ratio=4
    P_divider=6
    N=21
    Fo=560e6
    KVCO=90e6
    Icp=ICP
    Fp=x*Fc
    Fz=Fc/x
    k=(KVCO*Icp)/(2*pi*N)
    R1=(2*(pi)*Fc)/(k*(1-(1/(x**2))))
    wz=2*pi*Fz                       
    C1=1/(R1*wz)
    C2=C1/((x**2)-1)
    Ceq=C1*C2/(C1+C2)
    print(Ceq,C1,C2,R1)
    #VCO phase noise 
    w=np.linspace(2*pi*12e3,2*pi*20e6,50000)
    T_osc=control.tf([1,315.234e3],[1,0])
    T_osc=T_osc*103.9
    s=control.tf([1],[1,0,0])
    T_osc=T_osc*s
    T_osc

    #VCO phase noise 
    (mag, phase_rad, w) = control.bode(T_osc, w,dB=True,Hz=True,plot=False)
    for i in range(np.shape(w)[0]):
        if 10*np.log10(mag[i]) <= -145 :
            mag[i:]=mag[i]
            break
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('VCO phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Reference phase noise 
    T_ref=control.tf([1,16000*pi],[1,0])
    T_ref=T_ref*3.81115e-3
    s=control.tf([1],[1,0,0])
    T_ref=T_ref*s
    T_ref

    #Reference phase noise 
    (mag_ref, phase_rad_ref, w) = control.bode(T_ref, w,dB=True,Hz=True,plot=False)
    for i in range(np.shape(w)[0]):
        if 10*np.log10(mag_ref[i]) <= -150 :
            mag_ref[i:]=mag_ref[i]
            break
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_ref), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('Reference phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # transfer noise of loop filter is RC circuit noise 
    T_loopfilter=4*4.11e-21*R1

    #CP 
    T_CP_noise=control.tf([1,2*2e6*pi],[1,0])
    T_CP_noise=T_CP_noise*(5.10613e-24*(Icp))/(300e-6)
    (mag_CP_noise, phase_rad_CP_noise, w) = control.bode(T_CP_noise, w,dB=True,Hz=True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_CP_noise), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('CP phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # Feedforward for Feedback loop 
    Ts=control.tf([R1*C1*k,k],[R1*Ceq,1,0,0])
    Ts=Ts/((C1+C2))
    (mag_Feed, phase_rad_Feed, w) = control.bode_plot(Ts, w,dB=True, deg= True , margins=True,plot=False)


    # Feedback of the loop in the middle 
    #Closed loop 
    Bs=(Ts)/(1+(Ts))
    (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, deg= True,plot=False)

    # the overall closed loop
    Bs=(Ts*N)/((1+(Ts))*(R_ratio*P_divider))
    (mag_FB, phase_rad_FB, w) = control.bode_plot(Bs, w,dB=True, deg= True,plot=False)

    #Transfer function of different blocks 
    #Transfer function of VCO phase output to phase of VCO
    T_out_VCO=1/((1+(Ts))*P_divider)
    (mag_out_VCO, phase_out_VCO, w) = control.bode_plot(T_out_VCO, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_VCO)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    VCO_noise_output=mag*((mag_out_VCO)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(VCO_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('VCO noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of CP phase output to phase of CP
    T_out_CP=Ts*((2*pi)/Icp)*N
    T_out_CP=T_out_CP/(1+Ts)
    T_out_CP=T_out_CP/P_divider
    (mag_out_CP, phase_out_CP, w) = control.bode_plot(T_out_CP, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_CP)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(CP phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    CP_noise_output=mag_CP_noise*((mag_out_CP)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(CP_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('CP noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of CP phase output to phase of Reference
    T_out_ref=Bs
    (mag_out_ref, phase_out_ref, w) = control.bode_plot(T_out_ref, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_ref)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('(Ref phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    ref_noise_output=mag_ref*((mag_out_ref)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(ref_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('ref noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of different blocks 
    #Transfer function of LOOP_FILTER phase output to phase of Reference

    s_=control.tf([1,0],[1])
    T_out_loop_filter=((KVCO/s_)*(1/P_divider))/(1+Ts)
    (mag_out_loop_filter, phase_out_loop_filter, w) = control.bode_plot(T_out_loop_filter, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_loop_filter)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('loop_filter phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    loop_filter_noise_output=(4.112e-21)*4*R1*((mag_out_loop_filter)**2)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(loop_filter_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('loop Filter noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of R_Divider phase output to phase of Reference

    T_out_R_Divider=Bs*R_ratio
    (mag_out_R_Divider, phase_out_R_Divider, w) = control.bode_plot(T_out_R_Divider, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_R_Divider)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('R_Divider phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    # Divider transfer phase noise 
    T_Divider=control.tf([1,2*2e6*pi],[1,0])
    T_Divider=T_Divider*7.0710678e-17

    (mag_Divider, phase_rad_Divider, w) = control.bode(T_Divider, w,dB=True,Hz=True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(mag_Divider), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('Dividers phase noise')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    R_divider_noise_output=((mag_out_R_Divider)**2)*mag_Divider
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(R_divider_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('R Divider noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #Transfer function of N_Divider phase output to phase of Reference

    T_out_N_Divider=Bs*R_ratio
    (mag_out_N_Divider, phase_out_N_Divider, w) = control.bode_plot(T_out_N_Divider, w,dB=True, deg= True,plot=False)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10((mag_out_N_Divider)**2), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('N_Divider phase noise/ output)^2')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    N_divider_noise_output=((mag_out_N_Divider)**2)*mag_Divider
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(N_divider_noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('N Divider noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))


#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(2*mag_Divider), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('P Divider and Buffer noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    #calculation every noise at the output 
    noise_output=VCO_noise_output+CP_noise_output+ref_noise_output+loop_filter_noise_output+R_divider_noise_output+N_divider_noise_output+(2*mag_Divider)
#     plt.plot(np.log10(w/(2*pi)), 10*np.log10(noise_output), 'blue') 
#     plt.xlabel('Freq [Hz]') 
#     plt.ylabel('dBc/HZ') 
#     plt.title('noise at output')
#     plt.grid() 
#     plt.legend(labels=('mag',))

    from scipy import integrate
    jitter= np.trapz(noise_output, x=w/(2*pi))
    jitter=jitter*2
    jitter=(math.sqrt(jitter))/(2*pi*Fo)
    return(jitter*1e15)

In [50]:
jitter([1.00000000e-03, 7.50000000e+01, 2.59071638e+06])

3.3302713098997843e-13 1.921416035565107e-11 3.389010935671058e-13 24285.65086591471


181.2776614558112

In [41]:
import scipy.optimize as spo
bnds=((10e-6,1e-3),(60,75),(1e3,16e6))
intials=[300e-6,70,123e3]
res=spo.minimize(jitter,intials,method='Nelder-Mead',bounds=bnds)

5.853239719536989e-11 1.8826030976030157e-09 6.041063663020698e-11 3897.96091599704
6.145901705513838e-11 1.9767332524831663e-09 6.343116846171731e-11 3712.3437295209906
4.863114157673682e-11 2.313235224318745e-09 4.967546890489083e-11 3857.8726703902203
5.3090609701015766e-11 1.7075765057623723e-09 5.479422823601539e-11 4092.8589617968923
5.028044984923289e-11 2.0819727334464012e-09 5.152479361991706e-11 3870.2363274298973
4.631537293022557e-11 2.2030811660178535e-09 4.730997038561034e-11 3857.8726703902184
3.913658193685889e-11 2.159778813288894e-09 3.985884994749071e-11 4172.4410717883065
3.479914909341164e-11 2.0077536293616317e-09 3.541293962417992e-11 4450.233645627893
4.0818874201952124e-11 1.6582467266972627e-09 4.184901628126915e-11 4388.360247089856
3.7720111454748926e-11 1.4221397762853733e-09 3.874783942214439e-11 4655.605244445956
3.165309586671424e-11 1.8262405766398211e-09 3.2211396026880173e-11 4548.003930266685
2.8366801113618947e-11 1.6366362216606833e-09 2.8867137310

3.3568251391906083e-13 1.0393176042438521e-11 3.468863739893151e-13 28572.832105289104
2.869945916238989e-13 9.931975900766102e-12 2.955343588853757e-13 30032.360265005922
2.650316947085121e-13 9.257811691688613e-12 2.728426051526299e-13 31040.87641490117
2.18220551541451e-13 7.308334949701372e-12 2.2493697478102718e-13 34592.53620341675
3.188454687158806e-13 1.104557806265524e-11 3.283229523792739e-13 28676.502868292533
2.4711285992018e-13 8.369264851952797e-12 2.5463115871560165e-13 32410.87544504729
2.9855140528352314e-13 1.028403230221796e-11 3.074776595637882e-13 29569.971033062626
3.5862179411733586e-13 1.0765553199935311e-11 3.7097986454182605e-13 27782.216985630173
2.765830434326855e-13 9.759340503244271e-12 2.8465012521123885e-13 30352.824236379307
2.5146292002890255e-13 9.502245942647189e-12 2.5829840627853734e-13 31235.944767446028
2.7352291462273523e-13 9.189949363839573e-12 2.819135838274689e-13 30871.85756247435
1.9649995392993836e-13 8.306123833646872e-12 2.0126123909768

3.0268967246664786e-13 1.4244595035265298e-11 3.0926130205627627e-13 26858.415464728427
3.0169958955566573e-13 1.4218732839142174e-11 3.082399664237573e-13 26892.68715564493
3.023627497222398e-13 1.42361306276001e-11 3.0892402110417113e-13 26869.52324553196
3.018656523797841e-13 1.4223098970640438e-11 3.084112557759391e-13 26886.856615090066
3.021981200949158e-13 1.4231839659656902e-11 3.087541900073776e-13 26875.195772034273
3.0236274972223994e-13 1.4236130627600107e-11 3.089240211041713e-13 26869.523245531953
3.0440407380297567e-13 1.4302027145094369e-11 3.110239005889886e-13 26794.0253513926
3.01609126413088e-13 1.4213904963091646e-11 3.081478074621092e-13 26896.763033371233
3.020059689166906e-13 1.422797114648852e-11 3.0855543274905765e-13 26881.46472494085
3.017663092338809e-13 1.4221635655228686e-11 3.0830824602403045e-13 26889.859626874335
3.007882209869234e-13 1.4187813516375907e-11 3.073031910370261e-13 26927.293900944493
3.0155777726799026e-13 1.4213575396975927e-11 3.0809436

3.1758377624614144e-13 1.675304201136962e-11 3.237204749308923e-13 25469.279911859896
3.4530096923888485e-13 1.7940248803266633e-11 3.5207750307587204e-13 24517.69781755852
3.546966480516735e-13 1.8696113620496905e-11 3.6155597153404667e-13 24139.215802276824
2.8919885478169945e-13 1.5469863829460184e-11 2.9470822988131154e-13 26590.530180822334
3.0373829052265457e-13 1.6174285202420388e-11 3.0955138251260115e-13 25975.59332887371
3.0370304485223053e-13 1.6161655532754777e-11 3.095194035229277e-13 25972.681751399996
3.305122583071979e-13 1.7483667603822923e-11 3.3688066987293833e-13 24953.68835943902
3.288826467426883e-13 1.7200755499582238e-11 3.352935411509088e-13 25068.20980577418
3.2383643978646825e-13 1.711611573319823e-11 3.300815741069008e-13 25194.33807772657
3.2684955220965923e-13 1.729218022537428e-11 3.3314654882220606e-13 25071.05600809647
3.0861547365711177e-13 1.6267692116848155e-11 3.1458345743900805e-13 25825.664710093053
3.1400688056458853e-13 1.6566522199554453e-11 3.

3.3346618333660546e-13 1.9238904459631616e-11 3.393480726535283e-13 24269.849691478405
3.3203309300559e-13 1.91566626768654e-11 3.3788956812823376e-13 24322.024825930963
3.32347670220706e-13 1.9174774374619957e-11 3.3820970570233233e-13 24310.523750588804
3.3231341250189405e-13 1.9172855384525936e-11 3.381748258461934e-13 24311.757913909565
3.3202684528132517e-13 1.9156386333864664e-11 3.378831840350729e-13 24322.226014726453
3.325573846426801e-13 1.9186894941727363e-11 3.3842311255518494e-13 24302.850376936334
3.328199961801552e-13 1.920203792398388e-11 3.386903587082248e-13 24293.263152217478
3.32228127579447e-13 1.9168061469926575e-11 3.3808799726234533e-13 24314.836632406877
3.320708619665577e-13 1.9158987954818357e-11 3.3792795778438973e-13 24320.59358896517
3.328813613964394e-13 1.9205613273892518e-11 3.38752795446312e-13 24291.012450896083
3.3297301072940275e-13 1.9210958991580392e-11 3.388460432651121e-13 24287.65024503791
3.3330354681956404e-13 1.9230053386606857e-11 3.3918240

3.3302715977603555e-13 1.921416201647647e-11 3.3890112286089407e-13 24285.649816318408
3.3302724470532683e-13 1.921416691651274e-11 3.389012092881758e-13 24285.64671963009
3.330270886415634e-13 1.9214157912338914e-11 3.3890105047174564e-13 24285.652410021517
3.33027028567759e-13 1.9214154446351904e-11 3.389009893383541e-13 24285.654600431513
3.3302714747544635e-13 1.9214161306788063e-11 3.389011103433459e-13 24285.65026482209
3.3302718882766647e-13 1.9214163692624276e-11 3.3890115242494025e-13 24285.648757038907
3.3302710353269164e-13 1.9214158771489712e-11 3.389010656255249e-13 24285.651867061555
3.3302718521456024e-13 1.9214163484164374e-11 3.389011487481057e-13 24285.648888779848
3.3302711278480867e-13 1.9214159305295052e-11 3.3890107504083166e-13 24285.6515297111
3.330271764915064e-13 1.921416298088359e-11 3.3890113987119385e-13 24285.64920683951
3.3302712177239305e-13 1.9214159823838054e-11 3.389010841869399e-13 24285.651202006044
3.3302709491240003e-13 1.921415827413785e-11 3.389

In [42]:
print(res)

 final_simplex: (array([[1.00000000e-03, 7.50000000e+01, 2.59071638e+06],
       [1.00000000e-03, 7.50000000e+01, 2.59071638e+06],
       [1.00000000e-03, 7.50000000e+01, 2.59071638e+06],
       [1.00000000e-03, 7.50000000e+01, 2.59071638e+06]]), array([207.30961156, 207.30961156, 207.30961156, 207.30961156]))
           fun: 207.3096115607796
       message: 'Optimization terminated successfully.'
          nfev: 557
           nit: 286
        status: 0
       success: True
             x: array([1.00000000e-03, 7.50000000e+01, 2.59071638e+06])
